## Librairies

In [ ]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

## Algorithme

In [ ]:
# on choisi ici le visage admis
nom_visage_admis = "Raphael"

In [ ]:
with open('../data/noms.pkl', 'rb') as fh:
    noms = pickle.load(fh)
    
with open('../data/visages.pkl', 'rb') as fh:
    visages = pickle.load(fh)

print('Shape of visages matrix --> ', visages.shape)

# parcours de la liste "noms" pour sélectionner le visage de la personne admise uniquement
visages_admis = []
N = 0
for i, nom in enumerate(noms):
    if nom == nom_visage_admis:
        N += 1
        visages_admis.append(visages[i])
        
visages_admis = np.array(visages_admis)
print(visages_admis.shape)

In [ ]:
visages_admis = visages_admis.reshape(N, -1)

kmeans = KMeans(n_clusters = 2)
kmeans.fit(visages_admis)

## Exécution

In [ ]:
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0) # 0 pour 'built-in' caméra, 1 pour caméra externe

bal = 0 # Balise permettant de mettre fin au programme une fois le visage souhaité détecté
tmp = 0 # nombre de frame (grossièrement le 'temps') devant détecté le visage souhaité pour considérer que l'on est "Admis" 

while True:    
    ret, trame = camera.read()
    if ret == True:
        gris = cv2.cvtColor(trame, cv2.COLOR_BGR2GRAY)
        coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

        for (x, y, l, h) in coordonnees_visage:
            
            visage = trame[y:y + h, x:x + l, :]
            visage_redimensionne = cv2.resize(visage, (50, 50)).flatten().reshape(1,-1)
            
            texte = kmeans.predict(visage_redimensionne)
            
            #cv2.putText(trame, texte[0], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.rectangle(trame, (x, y), (x + l, y + l), (0, 0, 255), 2)
            
            #print(tmp)
            if texte[0] == 1 and tmp == 40:
                cv2.putText(trame, "Admis", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
                #kmeans.sleep(2.0)
                print("Admis")
                #bal = 1
                break
            elif texte[0] == 1:
                cv2.putText(trame, "Admis", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
                tmp += 1
            else:
                cv2.putText(trame, "Non Admis", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
                tmp = 0

        cv2.imshow('Reconnaissance faciale en temps réel', trame)
        
        if cv2.waitKey(1) == 27 or bal == 1:
            break
            
    else:
        print("erreur")
        break

cv2.destroyAllWindows()
camera.release()


## Visualisation des centroides

In [ ]:
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], marker='x', c='red', label='centroides')
plt.legend()
plt.show()